## Demo 2.2: How to expose a vcluster via a NodePort Service

##### Values.yaml preparation

In [19]:
! cat values.yaml

# Refer https://github.com/loft-sh/vcluster/blob/main/charts/k3s/values.yaml for full set of customisable options
 
syncer:
 extraArgs:
  # This is required to access vcluster using nodeport
  # It can be a comma separated list of master/worker node ips
  - --tls-san=172.18.0.3

# Service configurations
service:
  type: NodePort

##### Create vcluster with the customized values.yaml

In [11]:
! vcluster create vc1 -n host-namespace -f values.yaml

[info]   Creating namespace host-namespace
[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/818726559 --namespace host-namespace --install --repository-config='' --values /tmp/386942664 --values values.yaml
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


In [13]:
! kubectl get all -n host-namespace

NAME                                               READY   STATUS    RESTARTS   AGE
pod/coredns-5df468b6b7-n5rmv-x-kube-system-x-vc1   1/1     Running   0          15s
pod/vc1-0                                          2/2     Running   0          49s

NAME                                         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1         ClusterIP   10.96.112.239   <none>        53/UDP,53/TCP,9153/TCP   15s
service/vc1                                  NodePort    10.96.163.114   <none>        443:30676/TCP            49s
service/vc1-headless                         ClusterIP   None            <none>        443/TCP                  49s
service/vc1-node-kcd-vcluster-demo-worker3   ClusterIP   10.96.90.140    <none>        10250/TCP                15s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     49s


##### Accessing vcluster using NodePort Service

###### Use the nodeport corresponding to service/vc1 to download the kubeconfig

In [14]:
! vcluster connect vc1 --namespace host-namespace --server  https://172.18.0.3:30676

[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: ./kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig ./kubeconfig.yaml get namespaces`


In [15]:
! kubectl get ns --kubeconfig ./kubeconfig.yaml

NAME              STATUS   AGE
default           Active   45s
kube-system       Active   45s
kube-public       Active   45s
kube-node-lease   Active   45s


### Cleanup

In [17]:
! vcluster delete vc1 -n host-namespace
! kubectl delete ns host-namespace

[info]   Delete helm chart with helm delete vc1 --namespace host-namespace --kubeconfig /tmp/3254826469 --repository-config=''
[done] √ Successfully deleted virtual cluster vc1 in namespace host-namespace
[done] √ Successfully deleted virtual cluster pvc data-vc1-0 in namespace host-namespace
namespace "host-namespace" deleted


In [18]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   3h28m
kube-node-lease      Active   3h28m
kube-public          Active   3h28m
kube-system          Active   3h28m
local-path-storage   Active   3h28m
metallb-system       Active   3h28m
